# **Chapter 7 - Ensemble Learning and Random Forests**
- We all know that If we have a Question and we ask it's Answer from General Public and Accumulate Their Results most of the Times It would be better answer than and Experts Answer. This is known as **Wisdom of Crowd**.
- **Machine Learning** Also Functions and Behaves Similar to This if You Train A *model* that is *Expert* in a Certain Task and Someone Else Creates *1000* Relatively **Dumb Models** and then Culminate there Skills into one than There is a High Possibility His Model Overtaking Your Model in Every Aspect in the Language of Machine Learning it is known as **Ensemble Learning**.

## What We are Going to Learn
- Voting Classifier
- Bagging and Pasting
    - Bagging and Pasting in Scikit-Learn
    - Out-of-Bag Evaluation
- Random Patches and Random Subspaces
- Random Forests
    - Extra-Trees
    - Feature Importance
- Boosting
    - AdaBoost
    - Gradient Boosting
- Stacking Ensemble